# Normalized correlation count 2D (NCC 2D) Workflow
#### Author: Rodrigo dos Santos Maia Corrêa

#### The University of Texas at Austin, Austin, Texas USA

### Workflow Goal

Calculates Correlation sum, Correlation count and Normalized correlation count in two-dimensions for a point dataset within the area of interest.

### Workflow Steps 

1. **Data Loading** - basic data loading, checking and visualization
2. **Data Analysis** - Calculation of distances, azimuth, correlation sum and correlation count
3. **Randomizations** - Randomizes the dataset within the analyzed area
3. **Results** - Analysis of resulting calculations with graphs



### Functions

In [ ]:
#This part of the code was developed in colaboration with PGE Student Mahmood Shakiba

### 2D Correlation Sum in a cicular domain of radius R at an array of lenght scales r
def CorrSum_Random_2D_Array (r,R):
    CorrSum_rndm_array = np.zeros(len(r))
    for indx, ri in enumerate(r):        
        CorrSum_rndm_array[indx] = (np.pi*ri**2)/(np.pi*np.max(r)**2)
    return CorrSum_rndm_array

### 2D Correlation Count in a cicular domain of radius R at an array of lenght scales r
def CorrCount_Random_2D_Array (CorrSum):
    if W>0:
        CorrCount = CorrSum[(W*2):] - CorrSum[0:-(W*2)]
    else:
        CorrCount = CorrSum[1:] - CorrSum[0:-1]
    return CorrCount

# 1. Data Loading and Settings

#### Imports packages:

In [ ]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
from scipy import stats                   # summary statistics
import datetime
import math
import scipy.integrate as integrate              #for use of integrals
from scipy.spatial.distance import pdist         #spatial statistics package
from scipy.spatial.distance import squareform    #spatial statistics package

In [ ]:
os.chdir("")             # set the working directory

#### Define general settings for the workflow:

In [ ]:
nspl=100 # <------------Define number of length scales to be analysed
W=2 # <-------------------Define Windowing (similar to smoothing)
N_montecarlo=100 # <------------------- Define the number of montecarlo randomizations
grad_scale='log' # <---------------Define the type of graduation for length scale ('lin' for linear or 'log' for logarithmic)
Load_trace_map='yes' # <--------------Define if the trace map will be loaded for visualization (Use 'yes' to load fracture trace map)
Weight='yes' #<-----------------Define if weighted normalized correlation count is calculated (Use 'yes' or 'no')
Wf=5 #<-----------------Define the weighting factor (exagerates the weighting by the power of the factor)
Azimuthal='yes' #<-----------------Define if Azimuthal analysis is calculated (Use 'yes' or 'no')
nazi=24  ### <------------------------------SET NUMBER OF AZIMUTHS (suggested is 24 azimuths)

#### Define the circular analysis area:

In [ ]:
#Define circular area of interest (radius and center coordinates)
cr=  #Circle radius
ccx=   #Circle center x-position
ccy= #Circle center y-position

#### Loads fracture barycenter point set with length and azimuth information:

In [ ]:
df = pd.read_csv('', sep=',')# <-----------------Define fracture barycentre data file
df.head(n=5)

### Transforms azimuth information to pole

In [ ]:
pole=np.zeros((len(df['AZ'])))
for i in range(0,len(df['AZ'])):
    if df['AZ'][i]>=0 and df['AZ'][i]<=90:
        pole[i]=df['AZ'][i]+90
    if df['AZ'][i]>90 and df['AZ'][i]<=180:
        pole[i]=df['AZ'][i]-90
    if df['AZ'][i]>180 and df['AZ'][i]<=270:
        pole[i]=df['AZ'][i]-90
    if df['AZ'][i]>270 and df['AZ'][i]<=360:
        pole[i]=df['AZ'][i]-270

#### Creates a table with fracture trace end points coordinates, ID and size

In [ ]:
#Create fracture traces end positions
Xt1=(df['X']+np.sin(np.radians(df['AZ']))*df['L']/2)
Xt2=(df['X']-np.sin(np.radians(df['AZ']))*df['L']/2)
Yt1=(df['Y']+np.cos(np.radians(df['AZ']))*df['L']/2)
Yt2=(df['Y']-np.cos(np.radians(df['AZ']))*df['L']/2)

In [ ]:
#Organize fracture trace end positions with their respective ID and number of points (SZ)
Xt=np.asarray(list(sum(zip(Xt1, Xt2), ())))
Yt=np.asarray(list(sum(zip(Yt1, Yt2), ())))
IDt=np.asarray(list(sum(zip(df['ID'], df['ID']), ())))
Sz=np.linspace(2,2,(df['L'].size)*2)

In [ ]:
#Transforms all trace information in a table
Random_traces=np.vstack((np.vstack((np.stack((Xt,Yt)),IDt)),Sz))
dx=pd.DataFrame(Random_traces.T, columns=['X', 'Y', 'ID', 'SZ'])

### Plots visualization maps

In [ ]:
#Plot fracture traces and barycenters
#Barycenters out of the circular analysis area MUST BE DELETED.
plt.figure(figsize=(20,20))
if Load_trace_map==('yes'):
    for i in range(1, len(dx.ID)):
        if (i-1)==0:
                plt.plot(dx.X[i-1:i+int(dx.SZ[i-1]-1)], dx.Y[i-1:i+int(dx.SZ[i-1]-1)], 'r-')
        if (i-1)-(dx.SZ[i-1])<0:
                plt.plot(dx.X[i-int(dx.SZ[i-1]):i-1], dx.Y[i-int(dx.SZ[i-1]):i-1], 'r-')
        if dx.ID[i]!=dx.ID[i-1]:
            plt.plot(dx.X[i:i+int(dx.SZ[i])], dx.Y[i:i+int(dx.SZ[i])], 'r-')
        
plt.plot(df.X, df.Y, 'bo', label='Fracture barycenter', markersize=10)
plt.grid(b=True, which='major', color='#666666', linestyle='-', alpha=0.2)
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
ax = plt.gca()
ax.set_aspect(1)
circ=plt.Circle((ccx,ccy), cr, color='k', fill=False, linewidth=5)
ax.add_artist(circ)
#plt.minorticks_on()
ax.set_xlim((ccx-cr),(ccx+cr))
ax.set_ylim((ccy-cr),(ccy+cr))
plt.xlabel('Position X(m)',size=18)
plt.ylabel('Position Y(m)',size=18)
plt.legend(loc='upper right', fontsize=18)

# 2. Data Analysis

In [ ]:
#Creates some of the working variables
X=df['X']
Y=df['Y']
ID=df['ID']
nmax=len(ID)

In [ ]:
# Calculates distances between barycenters:
arr = np.stack((X, Y), axis=1)
dist=squareform(pdist(arr))

#### Creates length scale values:

In [ ]:
if 'log' in grad_scale:
    lscale=np.logspace(np.log10(np.min(dist[np.nonzero(dist)])),np.log10(cr*2),nspl) #logarithmic length scales #Builds the length scales that will be plotted and serve as reference to the correlation count and sum
    print('Logarithmic scale')
if grad_scale==('lin'):
    lscale=np.linspace((np.min(dist[np.nonzero(dist)])),cr*2,nspl) #linear length scales
    print('Linear scale')

#### Creates the translational edge correction:

In [ ]:
### Formula obtained from appendix B in the book Statistical Analysis and modelling of Spatial Point Patterns -  Illian (2008)
v=np.zeros((nmax,nmax))
v=1/((2*(cr**2)*np.arccos(dist/(2*cr)))-((dist/2)*(4*(cr**2)-(dist**2))**(1/2)))

#### Creates angle and length weigthing matrix:

In [ ]:
if Weight==('yes'):
    anw=np.zeros((nmax,nmax)) #Matrix holding the angle weights; Each element is filled below as an element-wise length x length x |cosine| multiplication
    lenw=np.zeros((nmax,nmax)) #Matrix holding the length weights; Each element is filled below by an element-wise length x length multiplication
    
    k=0
    j=0
    i=0
    for i in range(0,nmax):
        for j in range(0,nmax):
            if ID[i]!=ID[j]:
                anw[i,j]=(np.minimum((np.abs(pole[i]-pole[j])),(180-np.abs(pole[i]-pole[j]))))**Wf
                lenw[i,j]=(((df['L'][i])+(df['L'][j]))/2)**Wf
                

#### Calculates Correlation sum:

In [ ]:
corr_sum=np.zeros((np.size(lscale)))
count=0
if Weight==('yes'):
    corr_suma=np.zeros((np.size(lscale)))
    counta=0
    corr_suml=np.zeros((np.size(lscale)))
    countl=0
    
k=0
i=0
j=0
while k<np.size(lscale): #Counts how many spacings exists below a certain length scale value, that is corr_sum
    count=(np.sum(v*[np.where((dist < lscale[k]) & (dist != 0), 1, 0)]))
    corr_sum[k]=count #saves the corrsum value in the position k for each analyzed length scale
    count=0
    if Weight==('yes'):
        counta=np.sum(v*anw*[np.where((dist < lscale[k]) & (dist != 0), 1, 0)])
        corr_suma[k]=counta #saves the corrsum value in the position k for each analyzed length scale
        counta=0
        
        countl=np.sum(v*lenw*[np.where((dist < lscale[k]) & (dist != 0), 1, 0)])
        corr_suml[k]=countl #saves the corrsum value in the position k for each analyzed length scale
        countl=0
        
        
    k=k+1
k=0
corr_sum=((corr_sum)/(np.sum(v)))
if Weight==('yes'):
    corr_suma=((corr_suma)/(np.sum(v*anw))) #Divided by summation of anwmin to normalized between 0 and 1
    corr_suml=((corr_suml)/(np.sum(v*lenw))) #Divided by summation of lenwmin to normalize between 0 and 1
    

#### Calculates Slope of Correlation sum:

In [ ]:
slope_csum=np.zeros((np.size(corr_sum)-1))

if W>0:
    slope_csum=(np.log(corr_sum[(W*2):])-np.log(corr_sum[0:-(W*2)]))/(np.log(lscale[(W*2):])-np.log(lscale[0:-(W*2)]))
else:
    slope_csum=(np.log(corr_sum[1:])-np.log(corr_sum[0:-1]))/(np.log(lscale[1:])-np.log(lscale[0:-1]))

### Calculates azimuthal correlation analysis:

#### Sets azimuthal tolerance and azimuthal directions

In [ ]:
if Azimuthal==('yes'):
    azitol=360/nazi/2
    azidir=np.zeros(nazi)
    for i in range(0,(nazi)):
        azidir[i]=(360/nazi)*(i+1)-(360/nazi)

In [ ]:
if Azimuthal==('yes'):
    #Build azimuth matrix
    azi=np.zeros((nmax,nmax))
    for i in range(0,nmax):
        for j in range(0,nmax):
            if X[j]>=X[i] and Y[j]>=Y[i]:
                azi[i,j]=np.degrees(np.arctan((X[j]-X[i])/(Y[j]-Y[i])))
            if X[j]>=X[i] and Y[j]<Y[i]:
                azi[i,j]=np.degrees(np.arctan((Y[i]-Y[j])/(X[j]-X[i])))+90
            if X[j]<X[i] and Y[j]<Y[i]:
                azi[i,j]=np.degrees(np.arctan((X[i]-X[j])/(Y[i]-Y[j])))+180
            if X[j]<X[i] and Y[j]>=Y[i]:
                azi[i,j]=np.degrees(np.arctan((Y[j]-Y[i])/(X[i]-X[j])))+270

In [ ]:
if Azimuthal==('yes'):
    distaz=np.zeros((nazi,nmax,nmax))

    for k in range (0,nazi):
        mask=np.zeros((nmax,nmax))
        mask=np.where(((azi<(azidir[k]+azitol)) & (azi>(azidir[k]-azitol)))|((azi>(azidir[k]-azitol+360)) & (azi<360)),1,0) 
        distaz[k,:,:]=dist*mask

In [ ]:
if Azimuthal==('yes'):
    corr_sumaz=np.zeros((nazi,np.size(lscale)))
    count=0
    k=0
    i=0
    for i in range(0,nazi):
        while k<np.size(lscale): #Counts how many spacings exists below a certain length scale value, that is corr_sum
            count=(np.count_nonzero(np.where(distaz[i] < lscale[k], distaz[i], 0)))
            corr_sumaz[i,k]=count #saves the corrsum value in the position k for each analyzed length scale
            count=0
            k=k+1
        k=0
        i=i+1
    corr_sumaz=((corr_sumaz)/(np.max(corr_sumaz)))



#### Calculate Correlation count with windowing:

In [ ]:
if W>0:
    corr_count=corr_sum[(W*2):]-corr_sum[0:-(W*2)]
    if Weight==('yes'):
        corr_counta=corr_suma[(W*2):]-corr_suma[0:-(W*2)]
        corr_countl=corr_suml[(W*2):]-corr_suml[0:-(W*2)]
                
    if Azimuthal==('yes'):
            corr_countaz=corr_sumaz[:,(W*2):]-corr_sumaz[:,0:(-W*2)]
else:
    corr_count=corr_sum[1:]-corr_sum[0:-1]
    if Weight==('yes'):
        corr_counta=corr_suma[1:]-corr_suma[0:-1]
        corr_countl=corr_suml[1:]-corr_suml[0:-1]
                
    if Azimuthal==('yes'):
            corr_countaz=corr_sumaz[:,1:]-corr_sumaz[:,0:-1]

# 3. Randomizations

#### Calculates monte carlo randomizations using homogeneus poisson point process: 

In [ ]:
r=0

all_corr_sum_rand=np.zeros(((np.size(corr_sum)),N_montecarlo))
all_corr_count_rand=np.zeros(((np.size(corr_count)),N_montecarlo))
            
if Azimuthal==('yes'):  
    all_corr_sum_randaz=np.zeros((nazi,(np.size(corr_sumaz[0])),N_montecarlo))
    all_corr_count_randaz=np.zeros((nazi,(np.size(corr_countaz[0])),N_montecarlo))

while r<N_montecarlo: #Builds the random position of the points and distance matrix for the randomizations
#Simulates random points in the circular domain area
    pirand=np.random.rand(nmax,1)*2*(math.pi)
    radiusrand=np.sqrt(np.random.rand(nmax,1))*cr
       
    Xr=np.where((pirand>=0) & (pirand<((math.pi)/2)), ((ccx)+((np.sin(pirand))*radiusrand)), np.where((pirand>=((math.pi)/2)) & (pirand<(math.pi)), ((ccx)+((np.cos(pirand-((math.pi)/2)))*radiusrand)), np.where((pirand>=(math.pi)) & (pirand<((math.pi)*(3/2))), ((ccx)-((np.sin(pirand-(math.pi)))*radiusrand)), np.where((pirand>=((math.pi)*(3/2))) & (pirand<=(2*(math.pi))), ((ccx)-((np.cos(pirand-((math.pi)*(3/2))))*radiusrand)), 0))))
    Yr=np.where((pirand>=0) & (pirand<((math.pi)/2)), ((ccy)+((np.cos(pirand))*radiusrand)), np.where((pirand>=((math.pi)/2)) & (pirand<(math.pi)), ((ccy)-((np.sin(pirand-((math.pi)/2)))*radiusrand)), np.where((pirand>=(math.pi)) & (pirand<((math.pi)*(3/2))), ((ccy)-((np.cos(pirand-(math.pi)))*radiusrand)),np.where((pirand>=((math.pi)*(3/2))) & (pirand<=(2*(math.pi))), ((ccy)+((np.sin(pirand-((math.pi)*(3/2))))*radiusrand)), 0))))
    Xrand=Xr[:,0]
    Yrand=Yr[:,0]
    
#Build the distance matrix for randomizations 
    arr_rand = np.stack((Xrand, Yrand), axis=1)
    dist_rand=squareform(pdist(arr_rand))
    
#Build the translational edge correction
    vr=np.zeros((nmax,nmax))
    vr=1/((2*(cr**2)*(np.arccos(dist_rand/(2*cr))))-((dist_rand/2)*(4*(cr**2)-(dist_rand**2))**(1/2)))                            

    #Build the corrsum matrix for randomizations 
    #Unweighted correlation sum
    corr_sum_rand=np.zeros(((np.size(lscale))))
    count=0
    if Weight==('yes'):
        #Angle weighted correlation sum
        corr_sum_randa=np.zeros(((np.size(lscale))))
        counta=0
        #Lenght weighted correlation sum
        corr_sum_randl=np.zeros(((np.size(lscale))))
        countl=0
       
    k=0
    i=0
    j=0
    while k<np.size(lscale): #random values - Counts how many spacings exists below a certain length scale value, that is corr_sum
        count=(np.sum(vr*[np.where((dist_rand < lscale[k]) & (dist_rand != 0), 1, 0)]))
        corr_sum_rand[k]=count #random values - saves the corrsum value in the position k for each analyzed length scale
        count=0
        if Weight==('yes'):
            counta=np.sum(vr*anw*[np.where((dist_rand < lscale[k]) & (dist_rand != 0), 1, 0)])
            corr_sum_randa[k]=counta #random values - saves the corrsum value in the position k for each analyzed length scale
            counta=0
            
            countl=np.sum(vr*lenw*[np.where((dist_rand < lscale[k]) & (dist_rand != 0), 1, 0)])
            corr_sum_randl[k]=countl #random values - saves the corrsum value in the position k for each analyzed length scale
            countl=0
            
           
            
        k=k+1
    k=0       
    corr_sum_rand=((corr_sum_rand)/(np.sum(vr)))
    if Weight==('yes'):
        corr_sum_randa=((corr_sum_randa)/(np.sum(vr*anw)))
        corr_sum_randl=((corr_sum_randl)/(np.sum(vr*lenw)))
             
     
    if Azimuthal==('yes'):
    #Build Azimuthal matrix for randomizations 
        azi_rand=np.zeros((nmax,nmax))
        i=0
        j=0
        for i in range(0,nmax):
            for j in range(0,nmax):
                if Xrand[j]>=Xrand[i] and Yrand[j]>=Yrand[i]:
                    azi_rand[i,j]=np.degrees(np.arctan((Xrand[j]-Xrand[i])/(Yrand[j]-Yrand[i])))
                if Xrand[j]>=Xrand[i] and Yrand[j]<Yrand[i]:
                    azi_rand[i,j]=np.degrees(np.arctan((Yrand[i]-Yrand[j])/(Xrand[j]-Xrand[i])))+90
                if Xrand[j]<Xrand[i] and Yrand[j]<Yrand[i]:
                    azi_rand[i,j]=np.degrees(np.arctan((Xrand[i]-Xrand[j])/(Yrand[i]-Yrand[j])))+180
                if Xrand[j]<Xrand[i] and Yrand[j]>=Yrand[i]:
                    azi_rand[i,j]=np.degrees(np.arctan((Yrand[j]-Yrand[i])/(Xrand[i]-Xrand[j])))+270

        #Build distance matrix per azimuth
        dist_randaz=np.zeros((nazi,nmax,nmax))
        k=0
        for k in range (0,nazi):
            mask_rand=np.zeros((nmax,nmax))
            mask_rand=np.where(((azi_rand<(azidir[k]+azitol)) & (azi_rand>(azidir[k]-azitol)))|((azi_rand>(azidir[k]-azitol+360)) & (azi_rand<360)),1,0) 
            dist_randaz[k,:,:]=dist_rand*mask_rand

        #Build correlation sum azimuthal
        corr_sum_randaz=np.zeros((nazi,np.size(lscale)))
        count=0
        k=0
        i=0
        for i in range(0,nazi):
            while k<np.size(lscale): #Counts how many spacings exists below a certain length scale value, that is corr_sum
                countaz=(np.count_nonzero(np.where(dist_randaz[i] <= lscale[k], dist_randaz[i], 0)))
                corr_sum_randaz[i,k]=countaz #saves the corrsum value in the position k for each analyzed length scale
                countaz=0
                k=k+1
            k=0
            i=i+1
        corr_sum_randaz=((corr_sum_randaz)/(np.max(corr_sum_randaz)))

#Build the corrcount matrix for randomizations        
    if W>0:
        corr_count_rand=corr_sum_rand[(W*2):]-corr_sum_rand[0:(-W*2)]
        if Weight==('yes'):
            corr_count_randa=corr_sum_randa[(W*2):]-corr_sum_randa[0:(-W*2)]
            corr_count_randl=corr_sum_randl[(W*2):]-corr_sum_randl[0:(-W*2)]
                       
        if Azimuthal==('yes'):
            corr_count_randaz=corr_sum_randaz[:,(W*2):]-corr_sum_randaz[:,0:(-W*2)]
    else:
        corr_count_rand=corr_sum_rand[1:]-corr_sum_rand[0:-1]
        if Weight==('yes'):
            corr_count_randa=corr_sum_randa[1:]-corr_sum_randa[0:-1]
            corr_count_randl=corr_sum_randl[1:]-corr_sum_randl[0:-1]
                        
        if Azimuthal==('yes'):
            corr_count_randaz=corr_sum_randaz[:,1:]-corr_sum_randaz[:,0:-1]
            
                                           
#Builds the matrix to store the results of all realizations                                                
    all_corr_sum_rand[:,r:r+1]=np.reshape(corr_sum_rand[:],(np.size(corr_sum),1))
    all_corr_count_rand[:,r:r+1]=np.reshape(corr_count_rand[:],(np.size(corr_count),1))
                   
    if Azimuthal==('yes'):
        all_corr_sum_randaz[:,:,r]=corr_sum_randaz
        all_corr_count_randaz[:,:,r]=corr_count_randaz
    
    
    Rper=str(int(((r+1)/N_montecarlo)*100)) + str('%') + str('Randomization Nr.')+ str(r+1)
    print(Rper, end='\r')
    
    r=r+1


#### Calculates statistical results from randomizations:

In [ ]:
#Calculates the P2.5, P50 and P97.5 of the randomizations of unweighted case
P50_ccount=np.percentile(all_corr_count_rand,50, axis=1)
P50_csum=np.percentile(all_corr_sum_rand,50, axis=1)
P97_ccount=np.percentile(all_corr_count_rand,97.5, axis=1)
P2_5_ccount=np.percentile(all_corr_count_rand,2.5, axis=1)   
       
if Azimuthal==('yes'):
    #Calculates the P2.5, P50 and P95 of the randomizations of azimuthal case
    P50_ccountaz=np.percentile(all_corr_count_randaz,50, axis=2)
    P97_ccountaz=np.percentile(all_corr_count_randaz,97.5, axis=2)
    P2_5_ccountaz=np.percentile(all_corr_count_randaz,2.5, axis=2)

#### Calculates randomizations with analytical solution:

In [ ]:
#This part of the code was developed in colaboration with PGE Student Mahmood Shakiba
R = cr;                             # domain radius
dmin = lscale[0];                   # minimum length scale
dmax = lscale[nspl-1];              # maximum length scale
dstep = nspl;                       # number of length scales


d = lscale     # array of length scales

CorrSum = CorrSum_Random_2D_Array (d,R)
CorrCount = CorrCount_Random_2D_Array(CorrSum)

#### Calculates normalized correlation count:

In [ ]:
#Creates the normalized correlation count for the unweighted case
ncorr_count=corr_count/CorrCount
P97_nccount=P97_ccount/CorrCount
P2_5_nccount=P2_5_ccount/CorrCount
P50_nccount=P50_ccount/CorrCount

if Weight==('yes'):
    #Creates the normalized correlation count for the angle weighted case
    ncorr_counta=corr_counta/CorrCount
    
    #Creates the normalized correlation count for the length weighted case
    ncorr_countl=corr_countl/CorrCount   
       
if Azimuthal==('yes'):
    #Creates the normalized correlation count for the angle weighted case
    ncorr_countaz=corr_countaz/P50_ccountaz
    ncorr_countaz97=corr_countaz/P97_ccountaz
    ncorr_countaz2_5=corr_countaz/P2_5_ccountaz

# 4. Results

#### Compares montecarlo and analytical unweighted solutions:

In [ ]:
#Plots Correlation sum
plt.subplot(121)
plt.plot(d,CorrSum,color = 'blue', label='Analytical')
plt.plot(lscale,P50_csum,label='Montecarlo', color = 'green')
plt.xlabel('Length scale (m)',size=18)
plt.ylabel('Correlation Sum',size=18)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

if grad_scale=='log':
    plt.yscale("log"); plt.xscale("log");

plt.legend(loc='lower right',fontsize=15)
plt.grid(True,'both')

#Plots Correlation count
plt.subplot(122)
if W>1:
    plt.plot(d[1+(W-1):-W],CorrCount,color = 'blue', label='Analytical')
    plt.plot(lscale[1+(W-1):-W],P50_ccount,label='Montecarlo', color = 'green')
elif W==1:
    plt.plot(d[1:-W],CorrCount,color = 'blue', label='Analytical')
    plt.plot(lscale[1:-W],P50_ccount,label='Montecarlo', color = 'green')
else:
    plt.plot(d[1:],CorrCount,color = 'blue', label='Analytical')
    plt.plot(lscale[1:],P50_ccount,label='Montecarlo', color = 'green')

plt.xlabel('Length scale (m)',size=18)
plt.ylabel('Correlation Count',size=18)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

if grad_scale=='log':
    plt.yscale("log"); plt.xscale("log");

plt.legend(loc='lower right',fontsize=15)
plt.grid(True,'both')


plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.4, hspace=0.4)
plt.show()

#### Plots Correlation sum:

In [ ]:
plt.figure(figsize=(15,7.5))
plt.plot(lscale,corr_sum,label='Unweighted', linewidth=2, color='k')
if Weight==('yes'):
    plt.plot(lscale,corr_suma,label='Angle weighted', linewidth=2, color='g')
    plt.plot(lscale,corr_suml,label='Length weighted', linewidth=2, color='r')
    
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.xlabel('Length scale (m)',size=18)
plt.ylabel('Correlation Sum',size=18)
plt.legend(fontsize=15)
 
if grad_scale=='log':
    plt.xscale("log")
    plt.yscale("log")
plt.show
plt.savefig(fname='Csum')

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7.5))

ax1.set_xlabel('Length scale (m)',size=18)
ax1.set_ylabel('Correlation Sum',size=18)
ax1.plot(lscale,corr_sum,label='Unweighted CSum', linewidth=2, color='k')
plt.xscale("log")
plt.yscale("log")
ax1.set_ylim(None,1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Slope',size=18)  # we already handled the x-label with ax1

if W>1:
    ax2.plot(lscale[1+(W-1):-W],slope_csum,label='Slope', linewidth=2, color='b')

elif W==1:
    ax2.plot(lscale[1:-W],slope_csum,label='Slope', linewidth=2, color='b')
    
else:
    ax2.plot(lscale[1:],slope_csum,label='Slope', linewidth=2, color='b')
    

ax2.set_ylim(0,3)

plt.legend(fontsize=15)
plt.minorticks_on() 
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()
plt.savefig(fname='slope_csum')

#### Plots Normalized Correlation count:

In [ ]:
plt.figure(figsize=(15,7.5))

if W>1:
    plt.plot(lscale[1+(W-1):-W],ncorr_count,label='Unweighted', color='k')
    plt.plot(lscale[1+(W-1):-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
    plt.plot(lscale[1+(W-1):-W],P2_5_nccount, color='k',linestyle='--')
    
elif W==1:
    plt.plot(lscale[1:-W],ncorr_count,label='Unweighted', color='k')
    plt.plot(lscale[1:-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
    plt.plot(lscale[1:-W],P2_5_nccount,color='k',linestyle='--')
    
else:
    plt.plot(lscale[1:],ncorr_count,label='Unweighted', color='k')
    plt.plot(lscale[1:],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
    plt.plot(lscale[1:],P2_5_nccount,color='k',linestyle='--')
    

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.minorticks_on()
plt.xlabel('Length scale (m)',size=18)
plt.ylabel('Normalized Correlation Count',size=18)
plt.legend(fontsize=15)

if grad_scale=='log':
    plt.xscale("log")
    plt.yscale("log")

plt.show
plt.savefig(fname='NCCU')

In [ ]:
if Weight==('yes'):
    plt.figure(figsize=(15,7.5))

    if W>1:
        plt.plot(lscale[1+(W-1):-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1+(W-1):-W],ncorr_counta,label='Angle weighted', linewidth=3, color='g')
        plt.plot(lscale[1+(W-1):-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1+(W-1):-W],P2_5_nccount, color='k',linestyle='--')
        
    elif W==1:
        plt.plot(lscale[1:-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:-W],ncorr_counta,label='Angle weighted', linewidth=3, color='g')
        plt.plot(lscale[1:-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1:-W],P2_5_nccount,color='k',linestyle='--')
        
    else:
        plt.plot(lscale[1:],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:],ncorr_counta,label='Angle weighted', linewidth=3, color='g')
        plt.plot(lscale[1:],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1:],P2_5_nccount,color='k',linestyle='--')
       
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.minorticks_on()
    plt.xlabel('Length scale (m)',size=18)
    plt.ylabel('Weighted Normalized Correlation Count',size=18)
    plt.legend(fontsize=15)

    if grad_scale=='log':
        plt.xscale("log")
        plt.yscale("log")

    plt.show
    plt.savefig(fname='NCCA')

In [ ]:
if Weight==('yes'):
    plt.figure(figsize=(15,7.5))

    if W>1:
        plt.plot(lscale[1+(W-1):-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1+(W-1):-W],ncorr_countl,label='Length weighted', linewidth=3, color='r')
        plt.plot(lscale[1+(W-1):-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1+(W-1):-W],P2_5_nccount, color='k',linestyle='--')
        
    elif W==1:
        plt.plot(lscale[1:-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:-W],ncorr_countl,label='Length weighted', linewidth=3, color='r')
        plt.plot(lscale[1:-W],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1:-W],P2_5_nccount,color='k',linestyle='--')
                     
    else:
        plt.plot(lscale[1:],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:],ncorr_countl,label='Length weighted', linewidth=3, color='r')
        plt.plot(lscale[1:],P97_nccount,label='Confidence limits (95%)', color='k',linestyle='--')
        plt.plot(lscale[1:],P2_5_nccount,color='k',linestyle='--')
        
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.minorticks_on()
    plt.xlabel('Length scale (m)',size=18)
    plt.ylabel('Weighted Normalized Correlation Count',size=18)
    plt.legend(fontsize=15)

    if grad_scale=='log':
        plt.xscale("log")
        plt.yscale("log")

    plt.show
    plt.savefig(fname='NCCL')

In [ ]:
if Weight==('yes'):
    plt.figure(figsize=(15,7.5))

    if W>1:
        plt.plot(lscale[1+(W-1):-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1+(W-1):-W],ncorr_counta,label='Angle weighted', linewidth=2, color='g')
        plt.plot(lscale[1+(W-1):-W],ncorr_countl,label='Length weighted', linewidth=2, color='r')
                 

    elif W==1:
        plt.plot(lscale[1:-W],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:-W],ncorr_counta,label='Angle weighted', linewidth=2, color='g')
        plt.plot(lscale[1:-W],ncorr_countl,label='Length weighted', linewidth=2, color='r')
                
    else:
        plt.plot(lscale[1:],ncorr_count,label='Unweighted', linewidth=1, color='k')
        plt.plot(lscale[1:],ncorr_counta,label='Angle weighted', linewidth=2, color='g')
        plt.plot(lscale[1:],ncorr_countl,label='Length weighted', linewidth=2, color='r')
                
    plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.minorticks_on()
    plt.xlabel('Length scale (m)',size=18)
    plt.ylabel('Normalized Correlation Count',size=18)
    plt.legend(fontsize=15)

    if grad_scale=='log':
        plt.xscale("log")
        plt.yscale("log")

    plt.show
    plt.savefig(fname='NCCquad')

#### Plots Azimuthal Correlation Sum in Polar plot:

In [ ]:
if Azimuthal==('yes'):
    aziplot=np.append(azidir,360)
    corrsum_plot= np.vstack((corr_sumaz, corr_sumaz[0,:]))
    l, theta = np.meshgrid(lscale, np.radians(aziplot))

In [ ]:
#BUILD THE CORRELATION SUM AZIMUTHAL PLOT / POLAR PLOT
if Azimuthal==('yes'):
    vmin=0
    vmax=1
    levelscf = np.linspace(vmin, vmax, 100)
    fig1, ax = plt.subplots(subplot_kw=dict(projection='polar'),figsize=(30,30))
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)

    ax.set_title('2D correlation Sum', fontsize=40)
    contourf=ax.contourf(theta, l, corrsum_plot,cmap='coolwarm',levels=levelscf,vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(contourf)
    cbar.ax.tick_params(labelsize=30)
    ax.tick_params(labelsize=30)
    ax.grid(color='k')
    plt.show()

In [ ]:
if Azimuthal==('yes'):
    ncorrcount_plot= np.vstack((ncorr_countaz, ncorr_countaz[0,:]))
    ncorrcount97_plot= np.vstack((ncorr_countaz97, ncorr_countaz97[0,:]))
    ncorrcount2_5_plot= np.vstack((ncorr_countaz2_5, ncorr_countaz2_5[0,:]))

    if W>1:
        l, theta = np.meshgrid(lscale[1+(W-1):-W], np.radians(aziplot))
    elif W==1:
        l, theta = np.meshgrid(lscale[1:-W], np.radians(aziplot))
    else:
        l, theta = np.meshgrid(lscale[1:], np.radians(aziplot))


In [ ]:
#BUILD THE NORMALIZED CORRELATION COUNT AZIMUTHAL PLOT / POLAR PLOT
if Azimuthal==('yes'):
    vmin=0
    vmax=2
    levelscf = np.linspace(vmin, vmax, 100)
    levelsc = -9999,1,9999
    fig1, ax = plt.subplots(subplot_kw=dict(projection='polar'),figsize=(30,30))
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_title('2D Normalized Correlation Count', fontsize=40)
    contourf=ax.contourf(theta, l, ncorrcount_plot,cmap='coolwarm',levels=levelscf,vmin=vmin, vmax=vmax, extend='both')
    contour=ax.contour(theta, l, ncorrcount97_plot,levels=levelsc, colors='r',linewidth=5)
    contourb=ax.contour(theta, l, ncorrcount2_5_plot,levels=levelsc, colors='b',linewidth=5)
    cbar = plt.colorbar(contourf)
    cbar.ax.tick_params(labelsize=30)
    ax.tick_params(labelsize=30)
    ax.grid(color='k',linewidth=2)

    plt.show()